# In this notebook we are going to cover the most fundamental concepts of tensors using tensorflow

## More specfifically, we're going to cover:
    1) Introduction to tensors
    2) Getting information from tensors
    3) manipulating tensors
    4) Tensors & NumPy
    5) using @tf.function (a way to speed up your python functions)
    6) Using GPUs with TensorFlow (or TPUs)
    7) Exercises to try yourself!

## Introduction to Tensors

In [15]:
# Import TensorFLow
import tensorflow as tf 
print(tf.__version__)

2.10.1


In [16]:
# import numpy 
import numpy as np
print(np.__version__)

1.21.1


In [17]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
print(scalar)

tf.Tensor(7, shape=(), dtype=int32)


In [18]:
# Check the number of dimensions of a tensor (ndim stands for number of dimentions)
scalar.ndim

0

In [19]:
# Create a vector 
vector = tf.constant([10,10])
print(vector)

tf.Tensor([10 10], shape=(2,), dtype=int32)


In [20]:
# Check the dimention of our vector
print(vector.ndim)

1


In [21]:
# Create a matrix (has more than 1 dimention)
matrix = tf.constant([[10,7],
                      [7, 10]])
print(matrix)

tf.Tensor(
[[10  7]
 [ 7 10]], shape=(2, 2), dtype=int32)


In [22]:
# Check dimention of matrix
print(matrix.ndim)

2


In [23]:
# Create another matrix with shorter memory
another_matrix = tf.constant([[10., 7.], 
                              [7., 10.], 
                              [8., 9.]], dtype=tf.float16) # specify the data type with dtype parameter
print(another_matrix) # notice dtype

tf.Tensor(
[[10.  7.]
 [ 7. 10.]
 [ 8.  9.]], shape=(3, 2), dtype=float16)


In [24]:
# Number of dimention of "another_matrix"
print(another_matrix.ndim)

2


In [25]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9], 
                       [10, 11, 12]], 
                      [[13, 14, 15],
                       [16, 17, 18]]])
print(tensor)

tf.Tensor(
[[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]

 [[13 14 15]
  [16 17 18]]], shape=(3, 2, 3), dtype=int32)


In [26]:
# Check the dimentions of the tensor
print(tensor.ndim)

3


# What we've created so far:
* Scalar: a signle number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a two-dimentional array of numbers
* Tensor: an n-dimentional array of numbers (where n can be any number, a 0-dimentional tensor is a scalar, a 1-dimentional tensor is a vector)

### Creating tensors with `tf.Variable`

In [27]:
# Create the same tensor as above but with tf.Variable()
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
print(changeable_tensor, unchangeable_tensor)

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)> tf.Tensor([10  7], shape=(2,), dtype=int32)


In [29]:
# Let's try to change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [30]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [31]:
# Let's try to change our unchangeable_tensor
unchangeable_tensor[0].assign(7)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

**Note:** Rarely in practice will you need to decide whether the use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating Random Tensors
Random tensors are tensors of some arbitrary size with random numbers.

In [32]:
# Create two random (but the same) tensors
# "seed" allows tensorflow to geneate random numbers using a ~flavor~, "x". Where "x" is the seed.
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducability
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Suffle the order of elements in a tensor
This is useful for situations like image classifiication of ramen or spaghetti. Say you have 10k images and the first 8k images are of ramen. Your classifier will become biased to ramen because your dataset has not been randomized.

In [33]:
# Shuffle a tensor (valuable for when want to shuffle your data so the inherent order doesnt affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]], dtype=int32)>

In [34]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42) # global level seed
tf.random.shuffle(not_shuffled, seed=42) # operational level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**Exersize:** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in th esame order, we've got to use the global level random seed as well as the operation level random seed.

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

In [35]:
# Here, we set a global seed. Now we do not get a new shuffle every time.
tf.random.set_seed(42) # global level random seed
print(tf.random.shuffle(not_shuffled, seed=42)) # operation level random seed

tf.Tensor(
[[10  7]
 [ 3  4]
 [ 2  5]], shape=(3, 2), dtype=int32)


### Other ways to make tensors
Very similar to numpy operators

In [36]:
# Create a tensor of all ones 
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [37]:
# Create a tensor of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn numpy arrays into tensors
The main difference betweeen TF tensors and NumPy arrays is that tensors can be ran on a GPU (much faster for numerical computing).

In [38]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a Numpy array between 1 and 25
numpy_A
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [39]:
# Turn any numpy array into a tensor by passing it through soemthing like tf.constant.
# You can also arange the shape of the tensor
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(3, 8))
C = tf.constant(numpy_A, shape=(2, 3, 4))
A, B, C

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

### Getting information from tensors
When dealing with tensors, you probably want to be aware of the following attributes:
* Shape - The length (number of elements) of each of the dimentions of a tensor
* Rank - The number of tensor dimentions. A scalar has rank 0, a vector has rank 1, a matrix has rank 2, a tensor has rank n.
* Axis or dimention - A particular dimention of a tensor.
* Size - The total number of items in the the tensor.

In [40]:
# Create a rank 4 tensor (4 dimentions)
rank_4_tensor = tf.zeros(shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [41]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [42]:
rank_4_tensor[0]
# notice the "shape" in the output

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [43]:
# Get various attributes of our tensor
print("Datatype of every element:",rank_4_tensor.dtype)
print("Number of dimentions (rank):",rank_4_tensor.ndim)
print("Shape of the tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis:", rank_4_tensor.shape[0])
print("Elements along the last axis:", rank_4_tensor.shape[-1])
print("Total number of elements in our temsor:", tf.size(rank_4_tensor)) # messier without the .numpy() at the end
print("Total number of elements in our temsor:", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimentions (rank): 4
Shape of the tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our temsor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our temsor: 120


### Indexing tensors 
Tensors can be indexed just like Python lists.

In [44]:
# Get the first two elements of each diemention of rank_4_tensor

rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [45]:
# Get the first element from each dimention from each index except for the final one
rank_4_tensor[:1, :1, :1, :] # The lone colon gives you the entire index

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [46]:
# Get the first element from each dimention from each index except for the 3rd one
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [47]:
# Create a rank 2 tensor (2 dimentions)
rank_2_tensor = tf.constant([[10,7],
                             [3, 4]])
rank_2_tensor, rank_2_tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>,
 2)

In [48]:
# Get last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [49]:
# Add in extra dimention to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # the three dots replace doing this: rank_2_tensor[:, :, tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [50]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # -1 means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [51]:
tf.expand_dims(rank_2_tensor, axis=0) # expand 0 axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

### Manipulating Tensors (Tensor Operations)
**Basic Operations**

i.e. '+', '-', '*', '/'

In [52]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], 
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [53]:
# Original tensor is unchanged...
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [54]:
# In order to permanently change the tensor you must do:
tensor = tensor + 10
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [55]:
# Subtraction works the same way
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [56]:
# We can use the tf function as well. Using the tf functions will use your GPU and the calculation will be must quicker
tf.math.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[200, 170],
       [130, 140]], dtype=int32)>

In [57]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

Tensor multiplication follows the same rules as matrix multiplication.

In [58]:
# Matrix multiplication in tensor flow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[20 17]
 [13 14]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]], dtype=int32)>

In [59]:
# Matrix multiplication with python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]], dtype=int32)>

In [60]:
tensor.shape

TensorShape([2, 2])

In [61]:
# Create a (3, 2) tensor 
X = tf.constant([[1, 2],
                 [3, 4], 
                 [5, 6]])
# create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10], 
                 [11, 12]])

In [62]:
# multiply the two same-sized tensors
tf.matmul(X, Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [63]:
# Let's change the shape of Y
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [64]:
# Try to matrix multiply A by reshaped Y
X @ tf.reshape(Y, (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [65]:
# Use the tensorflow tensor multiplication function now
tf.matmul(X, tf.reshape(Y, (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [66]:
# Reshape X and then multiply
tf.matmul(tf.reshape(X, (2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [67]:
# You can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [68]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

Matrix Multiplication is also refered to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [69]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [70]:
# Perform the dot prodcuct on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [71]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [72]:
# Persorm matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

Notice how the results for `tf.matmul` are dfferent when you use "reshape" vs "transpose"

In [73]:
# Check the values of Y, reshape Y, and transpose Y
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, (2, 3)))

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)
Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [74]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when performing matrix multiplication on two tensors, you will transpose (rather than reshape) one of the tensors

### Changing the data type of a tensor

In [75]:
# Create a new tensor with default data type (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [76]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [77]:
# Change from float32 to float16
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [78]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [79]:
# Change from float32 to float16
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [80]:
# Create a tensor with values between 0 and 100 with shape (2, 5, 5)
numpy_linear_tensor = np.arange(0, 100, dtype=np.int32) # create a Numpy array between -50 and 50
linear_tensor = tf.constant(numpy_linear_tensor, shape=(4, 5, 5))

# Create a random tensor with values betwen 0 and 100 of size 50
# random_tensor = tf.random.Generator.from_seed(7)
# random_tensor = random_tensor.uniform(shape=(4, 5, 5), minval=0.0, maxval=100)
random_tensor = tf.constant(np.random.randint(0, 100, size=50))

linear_tensor, random_tensor

(<tf.Tensor: shape=(4, 5, 5), dtype=int32, numpy=
 array([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]],
 
        [[25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44],
         [45, 46, 47, 48, 49]],
 
        [[50, 51, 52, 53, 54],
         [55, 56, 57, 58, 59],
         [60, 61, 62, 63, 64],
         [65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74]],
 
        [[75, 76, 77, 78, 79],
         [80, 81, 82, 83, 84],
         [85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94],
         [95, 96, 97, 98, 99]]], dtype=int32)>,
 <tf.Tensor: shape=(50,), dtype=int64, numpy=
 array([64, 79, 19, 80, 53, 73,  1, 77, 50, 79, 63, 48, 79, 95, 33, 71, 55,
        18, 24, 78, 31, 40, 91, 55,  0, 78, 52, 11, 30, 31, 41, 19, 67, 50,
        30, 10, 89, 37, 14, 31, 69, 54, 85,  0, 55, 62, 98, 36, 39, 77])>)

In [81]:
tf.size(random_tensor), random_tensor.shape, random_tensor.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [82]:
# Get the absolute values and show the shape
tf.abs(linear_tensor), linear_tensor.shape, tf.abs(random_tensor), random_tensor.shape

(<tf.Tensor: shape=(4, 5, 5), dtype=int32, numpy=
 array([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]],
 
        [[25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39],
         [40, 41, 42, 43, 44],
         [45, 46, 47, 48, 49]],
 
        [[50, 51, 52, 53, 54],
         [55, 56, 57, 58, 59],
         [60, 61, 62, 63, 64],
         [65, 66, 67, 68, 69],
         [70, 71, 72, 73, 74]],
 
        [[75, 76, 77, 78, 79],
         [80, 81, 82, 83, 84],
         [85, 86, 87, 88, 89],
         [90, 91, 92, 93, 94],
         [95, 96, 97, 98, 99]]], dtype=int32)>,
 TensorShape([4, 5, 5]),
 <tf.Tensor: shape=(50,), dtype=int64, numpy=
 array([64, 79, 19, 80, 53, 73,  1, 77, 50, 79, 63, 48, 79, 95, 33, 71, 55,
        18, 24, 78, 31, 40, 91, 55,  0, 78, 52, 11, 30, 31, 41, 19, 67, 50,
        30, 10, 89, 37, 14, 31, 69, 54, 85,  0, 55, 62, 98, 36, 39, 77])>,
 Tenso

Lets go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor 
* Get the sum of a tensor

In [83]:
# Get the minimum valued element of the tensors
tf.reduce_min(linear_tensor), tf.reduce_min(random_tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

In [84]:
# Get the max valued element of the tensors
tf.reduce_max(linear_tensor), tf.reduce_max(random_tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=99>,
 <tf.Tensor: shape=(), dtype=int64, numpy=98>)

In [85]:
# Get the mean of the tensor
tf.reduce_mean(linear_tensor), tf.reduce_mean(random_tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=49>,
 <tf.Tensor: shape=(), dtype=int64, numpy=50>)

In [86]:
# Get the sum of the tensor
tf.reduce_sum(linear_tensor), tf.reduce_sum(random_tensor)

(<tf.Tensor: shape=(), dtype=int32, numpy=4950>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2521>)

**Exersize:** With what we've just learned, find the variance and standard deviation of our `random_tensor` using Tensorflow methods.

In [87]:
# Change from int64 to float32
random_tensor_float = tf.cast(random_tensor, dtype=tf.float32)
random_tensor_float

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([64., 79., 19., 80., 53., 73.,  1., 77., 50., 79., 63., 48., 79.,
       95., 33., 71., 55., 18., 24., 78., 31., 40., 91., 55.,  0., 78.,
       52., 11., 30., 31., 41., 19., 67., 50., 30., 10., 89., 37., 14.,
       31., 69., 54., 85.,  0., 55., 62., 98., 36., 39., 77.],
      dtype=float32)>

In [88]:
# Find the variance
tf.math.reduce_variance(random_tensor_float)

<tf.Tensor: shape=(), dtype=float32, numpy=714.2436>

In [89]:
# Find the standard deviation
tf.math.reduce_std(random_tensor_float)

<tf.Tensor: shape=(), dtype=float32, numpy=26.725336>

### Find the positional maximum and minimum of a tensor
This means that we want to find the **index** with the largest/smallest value across axes of a tensor

In [90]:
# create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [91]:
# Find the positional maximum
tf.math.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [92]:
# index on our largest value position
F[tf.math.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [93]:
# find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [94]:
# Check for equality
F[tf.math.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [95]:
# Find the positional minmum
tf.math.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [96]:
# find the minimum using hte positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimentions)

In [97]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [98]:
# Find the dhape of G
G.shape

TensorShape([1, 1, 1, 1, 50])

In [99]:
# Apply the squeeze on G
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### One-hot encoding tensors

In [100]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be representing items such as red, green, blue, or purple

# One hot encde our list of indices 
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [101]:
# Specify custom values for one-hot encoding
tf.one_hot(some_list, depth=4, on_value="yo i love deep learnng", off_value="poopoo")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo i love deep learnng', b'poopoo', b'poopoo', b'poopoo'],
       [b'poopoo', b'yo i love deep learnng', b'poopoo', b'poopoo'],
       [b'poopoo', b'poopoo', b'yo i love deep learnng', b'poopoo'],
       [b'poopoo', b'poopoo', b'poopoo', b'yo i love deep learnng']],
      dtype=object)>

**Practice:** Create your own list of indices and then one-hot encode it. See what happens when you change the `depth` parameter, and then change the `on_value` and `off_value`

In [102]:
# create a list of indices
some_list_2 = [4, 2, 5, 3, 1] # list that represents a type of ice cream flavor
# One-hot encode our list of indices
tf.one_hot(some_list_2, depth=10, on_value="YES", off_value="")

<tf.Tensor: shape=(5, 10), dtype=string, numpy=
array([[b'', b'', b'', b'', b'YES', b'', b'', b'', b'', b''],
       [b'', b'', b'YES', b'', b'', b'', b'', b'', b'', b''],
       [b'', b'', b'', b'', b'', b'YES', b'', b'', b'', b''],
       [b'', b'', b'', b'YES', b'', b'', b'', b'', b'', b''],
       [b'', b'YES', b'', b'', b'', b'', b'', b'', b'', b'']],
      dtype=object)>

### Squaring, log, and sqaure root

In [103]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [104]:
# Square it
tf.math.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [105]:
# Square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [106]:
# find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy
TensorFlow interacts beautifully with NumPy arrays

In [107]:
# Create a tensor directly from a numpy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [108]:
# Convert our tensor back to a numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [109]:
# Convert tensor J to a numpy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [110]:
# The default data types of nupy and tf are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatype of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

# TensorFlow Fundamentals Exercises

**1.** Create a vector, scalar, matrix, and tensor with values of your choosing using `tf.constant()`.

In [111]:
# create a scalar
K = tf.constant(10)
# create a vector
L = tf.constant([12, 10])
# create a matrix
M = tf.constant([[12, 10],
                [10, 12]])
# create a tensor
N = tf.constant([[[10, 12], [2, 6]],
                 [[2, 6], [12, 10]]])
K, L, M, N

(<tf.Tensor: shape=(), dtype=int32, numpy=10>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([12, 10], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[12, 10],
        [10, 12]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
 array([[[10, 12],
         [ 2,  6]],
 
        [[ 2,  6],
         [12, 10]]], dtype=int32)>)

**2.** Find the shape, rank, and size of the tensors you created in **1**.

In [112]:
# Find the shape of tensors
K.shape, L.shape, M.shape, N.shape

(TensorShape([]),
 TensorShape([2]),
 TensorShape([2, 2]),
 TensorShape([2, 2, 2]))

In [113]:
# Find the rank of the tensors
K.ndim, L.ndim, M.ndim, N.ndim

(0, 1, 2, 3)

In [114]:
# find the size of the tensors
tf.size(K), tf.size(L), tf.size(M), tf.size(N)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>,
 <tf.Tensor: shape=(), dtype=int32, numpy=4>,
 <tf.Tensor: shape=(), dtype=int32, numpy=8>)

**3.** Create two tensors containing random values between 0 and 1 with shape `[5, 300]`.

In [115]:
O = tf.constant(tf.random.uniform(shape=[1500]), shape=(5, 300))
P = tf.constant(tf.random.uniform(shape=[1500]), shape=(5, 300))
O, P

(<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.68789124, 0.48447883, 0.9309944 , ..., 0.6920762 , 0.33180213,
         0.9212563 ],
        [0.27369928, 0.10631859, 0.6218617 , ..., 0.4382149 , 0.30427706,
         0.51477313],
        [0.00920248, 0.37280262, 0.8177401 , ..., 0.56786287, 0.49201214,
         0.9892651 ],
        [0.88608265, 0.08672249, 0.12160683, ..., 0.91770685, 0.72545695,
         0.8280058 ],
        [0.36690474, 0.9200133 , 0.9646884 , ..., 0.69012   , 0.7137332 ,
         0.2584542 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 300), dtype=float32, numpy=
 array([[0.7413678 , 0.62854624, 0.01738465, ..., 0.4851334 , 0.21059811,
         0.25082767],
        [0.10842848, 0.48783147, 0.8240961 , ..., 0.9204427 , 0.36046863,
         0.28176582],
        [0.7326695 , 0.46489418, 0.13622475, ..., 0.28130388, 0.63987684,
         0.9987265 ],
        [0.01447165, 0.7845044 , 0.33475304, ..., 0.56194997, 0.0209924 ,
         0.1740731 ],
        [0.90936

**4.** Multiply the two tensors you created in **3** using matrix multiplication.

In [116]:
# use tf.matmul() to multiply the two tensors
tf.matmul(O, tf.transpose(P))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[75.714005, 80.87824 , 78.32848 , 78.259705, 79.130585],
       [70.12708 , 72.09945 , 70.1678  , 73.24609 , 74.277405],
       [75.16    , 79.52858 , 76.74644 , 78.14265 , 77.28679 ],
       [77.113556, 75.401215, 72.79378 , 75.066376, 75.206535],
       [79.87284 , 83.40138 , 78.57373 , 79.025894, 81.82093 ]],
      dtype=float32)>

**5.** Multiply the two tensors using dot product.

In [117]:
# use dot product
tf.tensordot(O, tf.transpose(P), axes=1)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[75.714005, 80.87824 , 78.32848 , 78.259705, 79.130585],
       [70.12708 , 72.09945 , 70.1678  , 73.24609 , 74.277405],
       [75.16    , 79.52858 , 76.74644 , 78.14265 , 77.28679 ],
       [77.113556, 75.401215, 72.79378 , 75.066376, 75.206535],
       [79.87284 , 83.40138 , 78.57373 , 79.025894, 81.82093 ]],
      dtype=float32)>

**6.** Create a tensor with random values between 0 and 1 with shape `[224, 224, 3]`.

In [118]:
Q = tf.constant(tf.random.uniform(shape=[150_528]), shape=(224, 224, 3))
Q

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[7.4023080e-01, 3.3938193e-01, 5.6925058e-01],
        [4.4811392e-01, 2.9285502e-01, 4.2600560e-01],
        [6.2890387e-01, 6.9106102e-01, 3.0925727e-01],
        ...,
        [9.1063976e-04, 6.9863999e-01, 1.7180574e-01],
        [6.7542684e-01, 8.3492923e-01, 3.9038682e-01],
        [2.3664141e-01, 6.2239432e-01, 1.0117912e-01]],

       [[9.7064960e-01, 5.4594779e-01, 7.6819682e-01],
        [2.6893330e-01, 2.6959443e-01, 2.8982997e-01],
        [2.9215467e-01, 1.7108858e-01, 6.5597153e-01],
        ...,
        [9.5621395e-01, 5.4591870e-01, 5.3843534e-01],
        [9.8861516e-01, 1.5786767e-01, 6.1375093e-01],
        [7.2668612e-01, 6.1637163e-03, 1.6534305e-01]],

       [[6.4095867e-01, 7.6697862e-01, 3.0138540e-01],
        [1.5474892e-01, 1.7183411e-01, 2.7192724e-01],
        [3.1211805e-01, 6.1451709e-01, 4.3001354e-01],
        ...,
        [8.4168065e-01, 5.0247252e-01, 2.8834987e-01],
        [3.3173549e-01

**7.** Find the min and max values of the tensor you created in **6** along the first axis.

In [119]:
# find the minimum along the first axis
tf.reduce_min(Q, axis=0)

<tf.Tensor: shape=(224, 3), dtype=float32, numpy=
array([[2.03430653e-03, 4.37009335e-03, 3.90768051e-04],
       [3.51417065e-03, 7.98487663e-03, 3.59594822e-03],
       [6.48093224e-03, 2.49707699e-03, 8.29935074e-04],
       [1.56582594e-02, 7.75694847e-04, 9.87851620e-03],
       [5.94484806e-03, 1.92798376e-02, 5.68044186e-03],
       [1.93202496e-03, 4.99880314e-03, 6.75463676e-03],
       [7.42089748e-03, 8.29935074e-04, 2.86221504e-04],
       [4.54080105e-03, 4.18329239e-03, 9.62257385e-04],
       [9.15646553e-04, 4.49621677e-03, 3.66115570e-03],
       [1.78742409e-03, 4.46248055e-03, 7.77256489e-03],
       [3.64875793e-03, 3.26538086e-03, 7.45463371e-03],
       [5.15615940e-03, 2.73168087e-03, 1.84178352e-04],
       [3.78739834e-03, 2.19345093e-05, 2.04169750e-03],
       [1.40672922e-02, 1.87754631e-04, 2.32815742e-04],
       [8.25285912e-04, 2.10213661e-03, 1.35517120e-03],
       [4.73082066e-03, 7.74383545e-04, 1.39951706e-03],
       [3.03184986e-03, 1.41239166e-03

In [120]:
# find the max along the the first axis
tf.reduce_max(Q, axis=0)

<tf.Tensor: shape=(224, 3), dtype=float32, numpy=
array([[0.999496  , 0.9963497 , 0.9888816 ],
       [0.9986012 , 0.99141884, 0.9950535 ],
       [0.997959  , 0.97667074, 0.9986603 ],
       [0.99689543, 0.99890375, 0.9978107 ],
       [0.9857353 , 0.9985192 , 0.9993119 ],
       [0.9998385 , 0.99915946, 0.99280477],
       [0.9994391 , 0.99572384, 0.995831  ],
       [0.9947666 , 0.98579705, 0.99513376],
       [0.99427724, 0.99754584, 0.9998121 ],
       [0.99203503, 0.99756145, 0.9958385 ],
       [0.99305665, 0.9890835 , 0.99911284],
       [0.9945315 , 0.9968053 , 0.99750936],
       [0.99853754, 0.99238455, 0.9903239 ],
       [0.9986687 , 0.99331665, 0.9950056 ],
       [0.9967083 , 0.9971876 , 0.99862325],
       [0.99219584, 0.99877846, 0.98672354],
       [0.99661255, 0.9858984 , 0.99987435],
       [0.99535847, 0.9920274 , 0.981122  ],
       [0.9940791 , 0.9999418 , 0.9879633 ],
       [0.991364  , 0.9968468 , 0.9968897 ],
       [0.99629796, 0.99595034, 0.99858797],
     

**8.** Create a tensor with random values of shape `[1, 224, 224, 3]` then squeeze it to change the shape to `[224, 224, 3]`.

In [121]:
R = tf.constant(tf.random.uniform(shape=[150_528]), shape=(1, 224, 224, 3))
R_squeezed = tf.squeeze(R)
R_squeezed

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[8.0315602e-01, 4.9777734e-01, 3.7054038e-01],
        [9.1186738e-01, 6.3764203e-01, 1.8209696e-01],
        [6.3791955e-01, 2.7701473e-01, 4.2271137e-02],
        ...,
        [1.0830712e-01, 4.5979273e-01, 2.5716281e-01],
        [8.7138689e-01, 1.8434000e-01, 4.4757760e-01],
        [7.4110627e-02, 9.0852141e-01, 5.3693414e-01]],

       [[5.5596435e-01, 6.8776274e-01, 7.6051474e-02],
        [1.6737962e-01, 7.1785092e-01, 2.7642274e-01],
        [2.6995218e-01, 3.2203627e-01, 8.8224900e-01],
        ...,
        [4.8168826e-01, 5.0150025e-01, 8.6756039e-01],
        [4.1261053e-01, 1.2770486e-01, 5.8186901e-01],
        [2.5495613e-01, 3.9036548e-01, 9.8529553e-01]],

       [[8.0935180e-01, 1.9740558e-01, 3.5899937e-01],
        [1.1216915e-01, 9.1016293e-04, 3.6382091e-01],
        [5.1202202e-01, 3.9188230e-01, 8.8335538e-01],
        ...,
        [2.0133841e-01, 9.1663551e-01, 1.9890130e-01],
        [8.0388057e-01

**9.** Create a tensor with shape `[10]` using your own choice of values, then find the index which has the maximum value.

In [122]:
# create tensor
S = tf.constant(10*tf.random.uniform(shape=[10]))
S

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([9.583183 , 0.1680839, 3.156035 , 1.6013157, 7.148702 , 7.892921 ,
       1.1484027, 3.3310425, 2.1091413, 6.23293  ], dtype=float32)>

In [123]:
# find the positional maximum
tf.math.argmax(S)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

**10.** One-hot encode the tensor you created in **9**.

In [124]:
# convert the values of S into type: int32
S_int32 = tf.cast(S, dtype="int32")

# one-hot the list
S_int32_one_hot = tf.one_hot(S_int32, depth=10)

S_int32, S_int32_one_hot

(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([9, 0, 3, 1, 7, 7, 1, 3, 2, 6], dtype=int32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)>)

# TensorFlow Fundamentals Extra-curriculum

* Read though the list of TensorFlow APIs, pick one havent gone through in this notebook, reverse engineer it (write out the documentation code for youself) and figure out what it does. (https://www.tensorflow.org/api_docs/python/)

* Try to create a series of tensor functions to calculate your most recent grocery bill (it's okay if you dont use the names of items, just the price in numerical form).
    * How would you calculate your grocery bill for the month and for the year using tensors?

* Go through the TensorFlow 2.x Quick Start for Beginners tutorial (be sure to type out all of the code yourself, even if you dont understand it).
    * are there any functions we used in here that match what's used in there? Which are the same? Which haven't you seen before?
    
* Watch the video "What's a tensor?" - a great visual introduction to many of the concepts we've convered in this notebook.(https://www.youtube.com/watch?v=f5liqUk0ZTw)

### Read though the list of TensorFlow APIs, pick one havent gone through in this notebook, reverse engineer it (write out the documentation code for youself) and figure out what it does:(https://www.tensorflow.org/api_docs/python/)

I have picked `cumsum`. I picked it becuase it has a hilarious name. 

To compute the cumulative sum of the tensor `x` along `axis`, you can use the TensorFlow function `tf.cumsum()`.

In [125]:
# set seed and create a tensor
tf.random.set_seed(10)
T = tf.constant(np.arange(12.0))
U = tf.constant(tf.random.normal(shape=[3, 2, 2], mean=0, stddev=5))
T, U

(<tf.Tensor: shape=(12,), dtype=float64, numpy=array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])>,
 <tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
 array([[[-4.3789043 ,  1.6781845 ],
         [-1.7609812 , -1.515728  ]],
 
        [[-0.19414826,  4.8264914 ],
         [ 3.1269023 , 13.325119  ]],
 
        [[-2.3185322 , -4.6196594 ],
         [-5.0100517 , -9.748539  ]]], dtype=float32)>)

By default, this op performs an inclusive sumsum, which means that the first element of the input is identical to the first element of the output. For example:

In [126]:
# tf.cumsum([a, b, c]) # [a, a+b, a+b+c]
tf.cumsum(T)

<tf.Tensor: shape=(12,), dtype=float64, numpy=array([ 0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45., 55., 66.])>

In [127]:
# Using varying axis values
# axis=0
tf.cumsum(U, axis=0)

<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[-4.3789043,  1.6781845],
        [-1.7609812, -1.515728 ]],

       [[-4.5730524,  6.504676 ],
        [ 1.3659211, 11.809391 ]],

       [[-6.8915844,  1.8850164],
        [-3.6441307,  2.060852 ]]], dtype=float32)>

In [128]:
# Using varying axis values
# axis=1
tf.cumsum(U, axis=1)

<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ -4.3789043 ,   1.6781845 ],
        [ -6.1398854 ,   0.16245651]],

       [[ -0.19414826,   4.8264914 ],
        [  2.932754  ,  18.151611  ]],

       [[ -2.3185322 ,  -4.6196594 ],
        [ -7.3285837 , -14.368198  ]]], dtype=float32)>

In [129]:
# Using varying axis values
# axis=2
tf.cumsum(U, axis=2)

<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ -4.3789043 ,  -2.7007198 ],
        [ -1.7609812 ,  -3.276709  ]],

       [[ -0.19414826,   4.6323433 ],
        [  3.1269023 ,  16.45202   ]],

       [[ -2.3185322 ,  -6.9381914 ],
        [ -5.0100517 , -14.758591  ]]], dtype=float32)>

In [130]:
# By setting the `exclusive` kwarg to `True`, an exclusive cumsum is performed instead:
# tf.cumsum([a, b, c], exclusive=True) => [0, a, a+b]
tf.cumsum(T, exclusive=True)

<tf.Tensor: shape=(12,), dtype=float64, numpy=array([ 0.,  0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45., 55.])>

In [131]:
# By setting the `reverse` kwarg to `True`, the cumsum is performed in the opposite direction:
tf.cumsum(T, reverse=True)

<tf.Tensor: shape=(12,), dtype=float64, numpy=array([66., 66., 65., 63., 60., 56., 51., 45., 38., 30., 21., 11.])>

In [132]:
# This is more efficient than using separate `tf.reverse` ops. The `reverse` and `exclusive` kwargs can also be combined:
# tf.cumsum([a, b, c], exclusive=True, reverse=True) # [b+c, c, 0]
tf.cumsum(T, exclusive=True, reverse=True)

<tf.Tensor: shape=(12,), dtype=float64, numpy=array([66., 65., 63., 60., 56., 51., 45., 38., 30., 21., 11.,  0.])>

### Try to create a series of tensor functions to calculate your most recent grocery bill (it's okay if you dont use the names of items, just the price in numerical form).
How would you calculate your grocery bill for the month and for the year using tensors?

In [144]:
# Create a list that holds the prices of each grocery item. The idex of the tensor represents a different grocery item:
# [milk=0, eggs=1, bread=2, rice=3, chicken=4, broccoli=5, spinach=6, potatoes=7]
price_list = tf.constant([4., 3., 5., 4., 6., 2., 3., 1.]) # 1xn matrix
# Create a list that shows which items i have bought. If i buy three cartons of eggs, i must write the egg's index "1" into the list three times.
inventory = tf.constant([0, 1, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7, 7, 7]) # 1xm matrix
# One-hot the inventory list. Make the `depth` of the one-hot to be the length of `price_list`.
inventory_one_hot = tf.one_hot(inventory, depth=len(price_list)) # mxn matrix
# Reduce sum the one_hot list along the zero axis to get a 1xn matrix
inventory_one_hot_reduce_sum = tf.reduce_sum(inventory_one_hot, 0) # 1xn
# Do regular multiplication of inventory_one_hot_reduce_sum by price_list to get a 1xn vector. This 1xn vector represents the money spent per item index
cost_by_item = price_list * inventory_one_hot_reduce_sum
# Reduce sum the cost_by_item vector along the 0-axis to get the total cost
total_cost = tf.reduce_sum(cost_by_item)

total_cost



<tf.Tensor: shape=(), dtype=float32, numpy=43.0>

### Go through the TensorFlow 2.x Quick Start for Beginners tutorial (be sure to type out all of the code yourself, even if you dont understand it).

https://www.tensorflow.org/tutorials/quickstart/beginner

Are there any functions we used in here that match what's used in there? Which are the same? Which haven't you seen before?

This short introduction uses `Keras` to:
1. Load a prebuilt dataset
2. Build a neural network machine learning model that classifies images
3. Train this neural network
4. Evaluate the accuracy of the model

**Load a dataset**

Load and prepare the `MNIST dataset`. Convert the sample data from intigers to floating-point numbers.

MNIST dataset:http://yann.lecun.com/exdb/mnist/

In [146]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

**Build machine learning model**

Build a tf.keras.Sepuencial model by stacking layers.

In [161]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [162]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.1983535 ,  0.44595915, -0.1125943 , -0.52204055, -0.40230304,
         0.7669983 , -0.3609059 , -0.48670015,  0.74972606, -0.13778731]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilites* for each class: 

In [163]:
tf.nn.softmax(predictions).numpy()

array([[0.07461811, 0.14212343, 0.08129971, 0.05398445, 0.06085134,
        0.19592561, 0.06342327, 0.0559264 , 0.1925706 , 0.07927711]],
      dtype=float32)

**note:** It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the neural network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparaseCategorialCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [164]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the inital loss should be close to `-tf.math.log(1/10). ~= 2.3`.

In [165]:
loss_fn(y_train[:1], predictions).numpy()

1.6300203

Before you start training, configure and compile the model using Keras `Model.compile`. Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to the evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [166]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

**Train and evaluate your model**

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [167]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2956 - accuracy: 0.9151
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1449 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1098 - accuracy: 0.9661
Epoch 4/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0881 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0760 - accuracy: 0.9767


The `Model.evaluate` method checks the model's performance, usually on a vaidation set or a test set.

In [168]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0747 - accuracy: 0.9769 - 1s/epoch - 4ms/step


[0.07472169399261475, 0.9768999814987183]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more read the TensorFlow tutorials. 

https://www.tensorflow.org/tutorials/

In [169]:
probability_model = tf.keras.Sequential([
    model, 
    tf.keras.layers.Softmax()
])

In [170]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.5041726e-06, 8.0210514e-09, 1.4703243e-05, 1.7181002e-03,
        1.1737996e-09, 4.2345905e-06, 3.2119316e-12, 9.9797541e-01,
        3.7829450e-05, 2.4813737e-04],
       [5.4151519e-07, 1.2882352e-04, 9.9977046e-01, 9.6545846e-05,
        1.6344495e-14, 1.4044363e-06, 1.4563180e-07, 3.8891178e-11,
        2.0028260e-06, 2.2588998e-11],
       [1.2269422e-07, 9.9808466e-01, 4.4191300e-05, 1.8475812e-05,
        2.1572539e-05, 3.9405408e-05, 1.4434773e-05, 4.6893698e-04,
        1.3036665e-03, 4.5851548e-06],
       [9.9996316e-01, 2.3652227e-10, 1.0130745e-05, 1.1570456e-07,
        2.1015985e-07, 1.9682132e-06, 5.3774893e-06, 9.7400357e-07,
        6.1535381e-08, 1.7968214e-05],
       [1.0320092e-05, 5.9990333e-07, 3.1437590e-05, 6.0595546e-08,
        9.9162078e-01, 1.4882390e-07, 1.2255874e-06, 4.8108708e-05,
        3.4838413e-07, 8.2870713e-03]], dtype=float32)>